## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-11-23-39-33 +0000,nyt,"Does President Trump Want to Be Mayor, Too?",https://www.nytimes.com/2025/08/11/us/politics...
1,2025-08-11-23-39-30 +0000,latimes,Safe zones to be expanded around LAUSD schools...,https://www.latimes.com/california/story/2025-...
2,2025-08-11-23-36-51 +0000,nypost,Ohio man competes in triathlon wearing jorts a...,https://nypost.com/2025/08/11/us-news/ohio-man...
3,2025-08-11-23-36-42 +0000,nyt,At Least One Dead in U.S. Steel Plant Explosio...,https://www.nytimes.com/2025/08/11/us/us-steel...
4,2025-08-11-23-35-43 +0000,nypost,Trump nominates Heritage Foundation economist ...,https://nypost.com/2025/08/11/us-news/trump-no...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2318/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
1,trump,67
236,china,18
56,he,14
94,new,14
52,deal,11


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
63,2025-08-11-21-29-00 +0000,wsj,President Trump said he would deploy National ...,https://www.wsj.com/politics/policy/trump-dc-n...,148
100,2025-08-11-19-43-00 +0000,wsj,The Trump administration’s decision to charge ...,https://www.wsj.com/politics/national-security...,146
267,2025-08-11-01-00-00 +0000,wsj,News the jobs market was much weaker than prev...,https://www.wsj.com/economy/jobs-inflation-tru...,133
170,2025-08-11-14-54-28 +0000,nypost,"Trump takes control of DC police, deploying Na...",https://nypost.com/2025/08/11/us-news/trump-fe...,132
48,2025-08-11-22-10-03 +0000,startribune,Trump says he's placing Washington police unde...,https://www.startribune.com/trump-is-promising...,127


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
63,148,2025-08-11-21-29-00 +0000,wsj,President Trump said he would deploy National ...,https://www.wsj.com/politics/policy/trump-dc-n...
100,71,2025-08-11-19-43-00 +0000,wsj,The Trump administration’s decision to charge ...,https://www.wsj.com/politics/national-security...
267,45,2025-08-11-01-00-00 +0000,wsj,News the jobs market was much weaker than prev...,https://www.wsj.com/economy/jobs-inflation-tru...
41,45,2025-08-11-22-21-40 +0000,nyt,Who Were the Al Jazeera Journalists Killed in ...,https://www.nytimes.com/2025/08/11/world/middl...
189,42,2025-08-11-13-46-00 +0000,wsj,Colombian Presidential Hopeful Dies Two Months...,https://www.wsj.com/world/americas/colombian-p...
167,38,2025-08-11-15-02-46 +0000,nypost,Wild video shows Chinese ships crashing while ...,https://nypost.com/2025/08/11/world-news/chine...
166,38,2025-08-11-15-03-55 +0000,nyt,"Netflix Signs New, Looser Deal With Prince Har...",https://www.nytimes.com/2025/08/11/business/me...
27,36,2025-08-11-22-41-56 +0000,nypost,Zohran Mamdani blasts Cuomo plan to block priv...,https://nypost.com/2025/08/11/us-news/zohran-m...
154,35,2025-08-11-16-09-47 +0000,nypost,Trump says he’ll tell Putin to end Ukraine inv...,https://nypost.com/2025/08/11/world-news/trump...
29,34,2025-08-11-22-35-28 +0000,nypost,Slain NYC 14-year-old boy’s inconsolable mom w...,https://nypost.com/2025/08/11/us-news/angel-me...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
